In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict
from sklearn.metrics import accuracy_score
import json

## Part 1

``` In this part I have generated the word vocab. ```

In [2]:
f = open("data/train", "r")
data = f.read()

In [3]:
word_vocab = {}

In [4]:
# Generate the frequency dictionary for all the words.

complete_data = data.split('\n')
total_sentences = 0

for index in range(len(complete_data)):
    each_token = complete_data[index].split('\t')

    if each_token[0] != '':
        if each_token[1] not in word_vocab:
            word_vocab[each_token[1]] = 1
        else:
            word_vocab[each_token[1]] += 1
    else:
        total_sentences += 1

In [5]:
# Arrange all the word according to decreasing order of frequency.

word_vocab_ordered = OrderedDict()
word_vocab_ordered['<unk>'] = 0
for w in sorted(word_vocab, key = word_vocab.get, reverse = True):
    if word_vocab[w] <= 1:
        word_vocab_ordered['<unk>'] += 1
    else:
        word_vocab_ordered[w] = word_vocab[w] 

In [6]:
print("Word Vocab Size:", len(word_vocab_ordered) )

Word Vocab Size: 23183


In [7]:
# Generate the desired output format.
index = 0
result = ""
for k, v in word_vocab_ordered.items():
    result+= k + "\t" + str(index) + "\t" + str(v) + "\n"
    index += 1

In [8]:
with open("vocab.txt", "w") as text_file:
    text_file.write(format(result))

## Part 2

``` This task includes the generation of transition, emission and start probabilities.```

In [9]:
hmm_result = ""

In [10]:
pos_tagging_dict = OrderedDict()

start_state_dict = OrderedDict()
start_state_dict[complete_data[0].split('\t')[2]] = 1

sentence_start = 0
for index in range(len(complete_data)):
    each_token = complete_data[index].split('\t')
    if each_token[0] != '':
        if sentence_start == 1:
            start_state_dict[each_token[2]] = start_state_dict.get(each_token[2], 0) + 1
            sentence_start = 0

        pos_tagging_dict[each_token[2]] =  pos_tagging_dict.get(each_token[2], 0) + 1
    else:
        sentence_start = 1   

In [11]:
#Calculate the starting probabilites for all the tags with which the sentence begins.

start_prob = {}


for each_state, each_value in start_state_dict.items():
    start_prob[each_state] =  each_value / sum(start_state_dict.values())
            
   

In [12]:
len(start_prob)

41

In [13]:
#Generate state transition dictionary.

state_transition_dict = OrderedDict()

for index in range(len(complete_data)-1):
    if complete_data[index] == '':
        continue
    else:
        if complete_data[index+1] != '':
            state1 = complete_data[index].split('\t')
            state2 = complete_data[index+1].split('\t')

            key = str(state1[2]+" , "+state2[2])
            state_transition_dict[key] = state_transition_dict.get(key, 0) + 1
        else:
            continue
            
            


In [14]:
transition = {}

for each_state, each_value in state_transition_dict.items():
    transition[each_state] =  each_value / pos_tagging_dict[each_state.split(' , ')[0]]

In [15]:
len(transition)

1351

In [16]:
emission_dict = OrderedDict()

for index in range(len(complete_data)-1):
    if complete_data[index] == '':
        continue
    else:
        each_state = complete_data[index].split('\t')
        if each_state[1] not in word_vocab_ordered:
            key =  each_state[2] + " , <unk>"
            emission_dict[key] = emission_dict.get(key, 0) + 1
        else:
            key =  each_state[2] + " , " + each_state[1]
            emission_dict[key] = emission_dict.get(key, 0) + 1

In [17]:
emission = {}

for each_state, each_value in emission_dict.items():
    emission[each_state] =  each_value / pos_tagging_dict[each_state.split(' , ')[0]]

In [18]:
len(emission)

30303

In [19]:
hmm_result = '{ \n "transition" : ' + json.dumps(transition) + ', \n "emission" : ' + json.dumps(emission) + ', \n "start_prob" : ' + json.dumps(start_prob) + "\n }" 

In [20]:
with open("hmm.json", "w") as text_file:
    text_file.write(format(hmm_result))

## Part 3

``` Greedy Decoding in Hidden Markov Models ```

In [21]:
dev_data = open("data/dev", "r")
dev_data = dev_data.read()

In [22]:
test_data = open("data/test", "r")
test_data = test_data.read()

In [23]:
def greedy_decoding(observation, emission, transition, states, start_prob):

    sequence = []

    for i in range(len(observation)):
        best_state = None
        max_prob = float('-inf')

        for st in states:
            if i == 0:

                if st in start_prob:
                    if(st + " , " + observation[i]) in emission:
                        cur_prob = start_prob[st] * emission[st + " , " + observation[i]]
                    else:
                        cur_prob = 0
                else:
                    cur_prob = 0

                if cur_prob > max_prob:
                    best_state = st

                    max_prob = cur_prob
            else:
                if (st+" , "+observation[i]) in emission and (sequence[-1] +" , "+ st) in transition:
                    cur_prob = transition[sequence[-1] +" , "+ st] * emission[st+" , "+observation[i]]
                else:
                    cur_prob = 0

                if cur_prob > max_prob:
                    max_prob = cur_prob
                    best_state = st

        sequence.append(best_state)
        
    return sequence
    

In [24]:
observation = []
dev_result = []
gtotal_dev_sequence = []
states = list(pos_tagging_dict.keys())

for obs in dev_data.split('\n'):
    if obs == "":
        each_sequence = greedy_decoding(observation, emission, transition, states, start_prob)
        gtotal_dev_sequence += each_sequence
        observation = []
        continue
    
    if obs.split('\t')[1] not in word_vocab_ordered:
        observation.append('<unk>')
    else:
        observation.append(obs.split('\t')[1])
    dev_result.append(obs.split('\t')[2])

In [25]:
accuracy_score(dev_result, gtotal_dev_sequence)

0.9350297492562686

In [26]:
observation = []
gtotal_test_sequence = []
states = list(pos_tagging_dict.keys())

# Generate results for the test data.

for obs in test_data.split('\n'):
    if obs == "":
        each_sequence = greedy_decoding(observation, emission, transition, states, start_prob)
        each_sequence.append("''")
        gtotal_test_sequence += each_sequence 
        observation = []
        
        continue
    
    if obs.split('\t')[1] not in word_vocab_ordered:
        observation.append('<unk>')
    else:
        observation.append(obs.split('\t')[1])


In [27]:
test_data_complete = test_data.split('\n')
test_result_string = ""

for i in range(len(test_data_complete)):
    obs = test_data_complete[i].split('\t')
    if obs[0] != '':
        test_result_string += obs[0]+'\t'+obs[1]+'\t'+gtotal_test_sequence[i]+'\n'
    else:
        test_result_string+="\n"

In [28]:
with open("greedy.out", "w") as text_file:
    text_file.write(format(test_result_string))

## Part 4

``` Viterbi Decoding in Hidden Markov Models. ```

In [29]:
def viterbi_decoding(observation, emission, transition, states, start_prob):
    v_table = np.zeros((len(observation), len(states)), dtype = float)
    b_table = np.zeros((len(observation), len(states)), dtype=int)
    
    idx_pos = {tag: idx for idx, tag in enumerate(states)}
    pos_idx = {idx: tag for idx, tag in enumerate(states)}
    
    for st in range(len(states)):
        state = states[st]
        v_table[0, st] = start_prob.get(state, 0) * emission.get(state +" , "+observation[0] , 0) 
        b_table[0, st] = 0
        

    for i in range(1, len(observation)):
        for cur_st in range(len(states)):
            state = states[cur_st]
            word = observation[i]
            max_prob = 0
            max_state = -1
    
            for prev_st in range(len(states)):
                prev_state = states[prev_st]
                if prev_state + ' , ' + state in transition and state+" , "+word in emission:
                    cur_prob = v_table[i-1, prev_st] * transition[prev_state + ' , ' + state] * emission[state+" , "+word] 
                else:
                    cur_prob = 0
                
                if cur_prob >= max_prob:
                    max_prob = cur_prob
                    max_state = prev_st
                    
            b_table[i, cur_st] = max_state
            v_table[i, cur_st] = max_prob
                            
    best_sequence = []
    last_state = np.argmax(v_table[len(observation)-1, :])
    best_sequence.append(states[last_state])
    
    for j in range(len(observation) - 1, 0, -1):
        last_state = b_table[j, last_state]
        best_sequence.append(states[last_state])
    
    best_sequence.reverse()
    return best_sequence
        

In [30]:
observation = []
true_result = []
vdev_total_sequence = []
states = list(pos_tagging_dict.keys())

for obs in dev_data.split('\n'):
    if obs == "":
        each_sequence = viterbi_decoding(observation, emission, transition, states, start_prob)
        vdev_total_sequence+=each_sequence
        observation = []
        continue
    
    if obs.split('\t')[1] not in word_vocab_ordered:
        observation.append('<unk>')
    else:
        observation.append(obs.split('\t')[1])
    true_result.append(obs.split('\t')[2])

In [31]:
accuracy_score(true_result, vdev_total_sequence)

0.9473013174670634

In [32]:
observation = []
viterbi_total_test_sequence = []
states = list(pos_tagging_dict.keys())
index = 1

for obs in test_data.split('\n'):
    if obs == "":
        each_sequence = viterbi_decoding(observation, emission, transition, states, start_prob)
        each_sequence.append("''")
        viterbi_total_test_sequence += each_sequence 
        observation = []
        continue
    
    if obs.split('\t')[1] not in word_vocab_ordered:
        observation.append('<unk>')
    else:
        observation.append(obs.split('\t')[1])   

In [33]:
test_data_complete = test_data.split('\n')
vtest_result_string = ""

for i in range(len(test_data_complete)):
    obs = test_data_complete[i].split('\t')
    if obs[0] != '':
        vtest_result_string += obs[0]+'\t'+obs[1]+'\t'+viterbi_total_test_sequence[i]+'\n'
    else:
        vtest_result_string+="\n"

In [34]:
with open("viterbi.out", "w") as text_file:
    text_file.write(format(vtest_result_string))